# Galoisfeld

Alice nutzt eine Tauschchiffre über dem Körper $GF(2^4)$: Ein 4-Bit-Klartextzeichen $k$ wird zum 4-Bit-Geheimtextzeichen $c$ durch $c = k\cdot a + b (in GF(2^4))$ verschlüsselt. Dabei stellen $a$ und $b$ (aus $GF(2^4)$) den geheimen Schlüssel dar.

Rechnungen im Körper $GF(2^4)$ werden wie folgt durchgeführt:
Die Addition entspricht einem Bit-weisen XOR.
Für die Multiplikation wird ein 4-Bit-Zeichen als Polynom mit den entsprechenden Koeffizienten aufgefasst (z.B. $1101$ als $x^3+x^2+1$ und $0110$ als $x^2+x$), dann multipliziert (wobei die Koeffizienten modulo 2 berechnet werden) und modulo $g(x)=x^4+x+1$ genommen (Rest bei der Polynomdivision durch $g(x)$, auch hier alles modulo 2); das resultierende Polynom wird dann wieder als 4-Bit-Zeichen aufgefasst; bei $1101$ und $0110$ ergibt sich beispielsweise

$(x^3+x^2+1) \cdot (x^2+x) = x^5+x^3+x^2+x$

und $(x^5+x^3+x^2+x) : x^4+x+1 = x$, Rest $x^3$,

also: $1101 \cdot 0110 = 1000$.

Mallory beobachtet nun, dass das Klartextzeichen $1011$ in das Geheimtextzeichen $0111$ und das Klartextzeichen $0110$ in das Geheimtextzeichen $1001$ verschlüsselt wird.

Welchen Schlüssel, d.h. welche $a$ und $b$, nutzt Alice?

### Lösung

$0111_2 \equiv 1011_2 \cdot a + b \ GF(2^4)$

$1001_2 \equiv 0110_2 \cdot a + b \ GF(2^4)$

$\therefore \ (0111_2 + 1001_2) \equiv (1011_2+0110_2) \cdot a \ GF(2^4)$

$\therefore \ a \equiv (1011_2+0110_2)^{-1} \cdot (0111_2 + 1001_2) \ GF(2^4)$

$\therefore \ b \equiv 0111_2 + 1011_2 \cdot a \ GF(2^4)$

Die Lösung für $a^{-1}$ kann mittels des Euklidischen Algorithmus gefunden werden.

In [67]:
class GF:
    def __init__(self, val):
        self.val = val
    
    def __add__(self, other):
        s = self.val ^ other.val
        for i in range(3, -1, -1):
            if s > 0b10011<<i:
                s ^= 0b10011<<i
        return GF(s)
    
    def __mul__(self, other):
        result = 0
        for i in range(4):
            if other.val & (1<<i):
                result ^= self.val<<i
        for i in range(3, -1, -1):
            if result > 0b10011<<i:
                result ^= 0b10011<<i
        return GF(result)

    def __truediv__(self, other):
        result = 0
        divident = self.val
        for i in range(3, -1, -1):
            if divident > (divident ^ (other.val<<i)):
                divident ^= other.val<<i
                result ^= 1<<i
        return GF(result)
    
    def __eq__(self, other):
        return self.val == other.val
    
    def inverse(self):
        t, t_ = GF(0), GF(1)
        r, r_ = GF(0b10011), self
        while r_.val != 0:
            q = r / r_
            r, r_ = r_,  r + q * r_
            t, t_ = t_,  t + q * t_
        return (GF(1) / r) * t


p0 = GF(0b1011)
c0 = GF(0b0111)
p1 = GF(0b0110)
c1 = GF(0b1001)

a = (p0+p1).inverse()*(c0+c1)
b = p0 * a + c0

if p0*a+b == c0 and p1*a+b == c1:
    print(f"Test bestanden! Der Schlüssel lautet: A = {bin(a.val)[2:]} und B = {bin(b.val)[2:]}")

Test bestanden! Der Schlüssel lautet: A = 1101 und B = 1
